# Retention Call Inlight Drive 

In [1]:
import pandas as pd

SG = pd.read_csv('segement.csv')

In [2]:
#importing table 'BIGTABLE(Details)'and read as pandas.....
BT = pd.read_csv('Bigtable.csv',usecols = ['Full SFID','Contract Id','Type','Customer Signed By: Full Name','Customer Signed By: Phone','Customer: Secondary Phone number','Customer Warranty End Date','Price Book: Price Book Name'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
Light = pd.read_csv('Light_Report.csv',usecols = ['contract_id','Days_in_light','LTOperiod','type','generation','ContractCreation'])

In [4]:
join = pd.merge(Light, BT, how='left',left_on='contract_id', right_on='Full SFID')

In [5]:
join1 = pd.merge(join, SG[['Contract Id','Potential Grouping Other']], how='left', on='Contract Id')

In [6]:
join1['Customer Warranty End Date'] = pd.to_datetime(join1['Customer Warranty End Date'], format='%d/%m/%Y')

join1['ContractCreation'] = pd.to_datetime(join1['ContractCreation'], format='%Y/%m/%d')

join1['ContractCreation'] = pd.to_datetime(join1['ContractCreation']).dt.normalize()

In [7]:
import datetime as date

check = pd.to_datetime('01/06/2022')

def warranty(i):
    if i >= check:
        return 'Within Warranty'
    elif i == ' ':
        return 'N/A'
    else:
        return 'Outside Warranty'
    

join1['Warranty Status'] = join1['Customer Warranty End Date'].apply(warranty)

In [8]:
import numpy as np

filters = [
   (join1.Days_in_light == 1),
   (join1.Days_in_light == 2),
   (join1.Days_in_light == 3),
   (join1.Days_in_light == 5)
]
values = ['1 LightDays', '2 LightDays', '3 LightDays','5 LightDays']

join1["Light_Days"] = np.select(filters, values, default= join1['Days_in_light'])

In [9]:
join2 = join1[['Contract Id','Days_in_light','LTOperiod','Type','Customer Signed By: Full Name','Customer Signed By: Phone',
               'Customer: Secondary Phone number','type','ContractCreation','generation','Light_Days','Customer Warranty End Date',
               'Warranty Status','Potential Grouping Other']]

In [10]:
join2 = join2[((join2['Days_in_light'] == 1) | (join2['Days_in_light'] == 2) | (join2['Days_in_light'] == 3) | (join2['Days_in_light'] == 5)) & ((join2['Potential Grouping Other'] == 'Good') & (join2['Type'] == 'Production'))]


join2.columns = ['Contract Short Num','Consecutive No Activity Days','Eula_Free_to_Use_Days','Type','Customer Name',
               'Payer Number','Secondary Number','Contract Model Type','Contract Start Date','Generation','Light Days',
               'Warranty End Date','Warranty Status','Customer Segmentation']


In [11]:
join2.drop('Type', axis = 1, inplace=True)

In [12]:
join2.to_excel('Retention Call Inlight Drive.xlsx')

In [13]:
join2.groupby(['Generation','Light Days']).count()['Contract Short Num']

Generation  Light Days 
LEGACY      1 LightDays    182
            2 LightDays    179
            3 LightDays    209
            5 LightDays    219
UNIFIED     1 LightDays    131
            2 LightDays    176
            3 LightDays    228
            5 LightDays    298
Name: Contract Short Num, dtype: int64